In [ ]:
import numpy as np
import pandas as pd
import gurobipy as gp
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

RUNTIME = 600
kfolds = KFold(n_splits=10)
big_m = 100
# Direct Variable Selection – MIQP Problem
General form of Quadratic Optimization problem :

Objective function : $x^{T}Qx + c^{T}x$ <br>
such that <br> 
$ Ax \le b $<br>
$ x \ge 0 $
MIQL implementation has the below decision variables:
1. $m+1$ betas - $ \beta_{0}, \beta_{1} ... \beta_{m}$
2. $m$ variables for variable selection $ z_{1}, z_{2} ... z_{m}$   <br>

Total Constraint equations - 2m +1

Objective function for MIQP : $$ \beta^{T}(X^{T}X)\beta + (-2y^{T}X)\beta$$<br>
such that <br> 
$$ -z_{i}M < \beta_{i} < z_{i}M $$   $\forall  i \in [1,m]$<br>
$$ \sum_{i=1}^m z_{i} \le k$$

Comparing with the quadratic form we have :
$Q =(X^{T}X)$<br>
$c =(-2y^{T}X)$
### Find the best k using MIQP
lasso_d = {}
k_lst = []
sse_lst = []

for k in [5,10,15,20,25,30,35,40,45,50]:
    
    train = pd.read_csv('training_data.csv')
    X = np.array(train.iloc[0:,1:])
    y = np.array(train['y'])
    
    sse = 0
    
    # cross validation
    for train_index, holdout_index in kfolds.split(X):
        X_train, X_holdout = X[train_index], X[holdout_index]
        y_train, y_holdout = y[train_index], y[holdout_index]
        
        n = X_train.shape[0] # number of rows
        m = X_train.shape[1] # number of variables

        # quadratic term Q
        Q = np.zeros((2*m+1, 2*m+1))
        X_Q = np.zeros((n,m+1))
        X_Q[0:n, 0] = 1
        for i in range(n):        
            X_Q[i, 1:] = X_train[i,:]
        Q[:m+1,:m+1] = X_Q.T @ X_Q

        # linear term C
        C = np.zeros(2*m+1)
        C[:m+1] = -2*(y_train.T) @ X_Q
        
        # create A matrix
        A = np.zeros((2*m+1, 2*m+1))

        # big M constraint: weights-M*binary  <= 0
        np.fill_diagonal(A[:m, 1:m+1], 1) 
        np.fill_diagonal(A[:m, m+1:2*m+1], -big_m) 
        
        # big M constraint: weights+M*binary  >= 0
        np.fill_diagonal(A[m:-1, 1:m+1], 1) 
        np.fill_diagonal(A[m:-1, m+1:2*m+1], big_m) 

        # k stocks are chosen
        A[-1, m+1:] = 1

        sense = np.array(['<']*m + ['>']*m + ['<'])

        b = np.concatenate((np.zeros(2*m), [k]))

        lb = np.array([np.NINF]+[-big_m]*m+[np.NINF]*m)

        ndxMod = gp.Model()
        ndxMod_x = ndxMod.addMVar(len(Q),vtype=['C']*(m+1)+['B']*m, lb=lb) 
        ndxMod_con = ndxMod.addMConstrs(A, ndxMod_x, sense, b)
        ndxMod.setMObjective(Q, C, 0, sense=gp.GRB.MINIMIZE)
        ndxMod.Params.OutputFlag = 0
        ndxMod.setParam('TimeLimit', RUNTIME)
        ndxMod.optimize()
        
        # save the coefficients
        coef = ndxMod_x.x[:m+1]
        
        n = X_holdout.shape[0] # number of rows
        
        # X variables for each record in the holdout set
        X_Q = np.zeros((n,m+1))
        X_Q[0:n, 0] = 1
        for i in range(n):        
            X_Q[i, 1:] = X_holdout[i,:]
        
        # calculate the SSE for holdout set
        sse += np.transpose(X_Q @ coef - y_holdout) @ (X_Q @ coef - y_holdout)
        
    print(k,':',sse)
    lasso_d[sse] = k
    
    k_lst.append(k)
    sse_lst.append(sse)
    
best_k = lasso_d[min(k for k, v in lasso_d.items())] # save the best k
print('The best number of X variables is:', best_k)
### Plot the SSE
plt.figure(figsize=(8,5))
plt.bar(k_lst, sse_lst, label='SSE')
plt.title('Performance Evaluation')
plt.xlabel('Number of X Variables')
plt.ylabel('Sum of Square Errors')
plt.legend()
plt.show
### Fit the MIQP model on the entire training set with k=10
train = pd.read_csv('training_data.csv')

X_train = np.array(train.iloc[0:,1:])
y_train = np.array(train['y'])

n = X_train.shape[0] # number of rows
m = X_train.shape[1] # number of variables

# quadratic term Q
Q = np.zeros((2*m+1, 2*m+1))
X = np.zeros((n,m+1))
X[0:n, 0] = 1
for i in range(n):        
    X[i, 1:] = X_train[i,:]
Q[:m+1,:m+1] = X.T @ X

# linear term C
C = np.zeros(2*m+1)
C[:m+1] = -2*(y_train.T) @ X

# create A matrix
A = np.zeros((2*m+1, 2*m+1))

# big M constraint: weights-M*binary  <= 0
np.fill_diagonal(A[:m, 1:m+1], 1) 
np.fill_diagonal(A[:m, m+1:2*m+1], -big_m) 
        
# big M constraint: weights+M*binary  >= 0
np.fill_diagonal(A[m:-1, 1:m+1], 1) 
np.fill_diagonal(A[m:-1, m+1:2*m+1], big_m) 

# k stocks are chosen
A[-1, m+1:] = 1

sense = np.array(['<']*m + ['>']*m + ['<'])

b = np.concatenate((np.zeros(2*m), [best_k]))

lb = np.array([-big_m]*(m+1)+[0]*m)

ndxMod = gp.Model()
ndxMod_x = ndxMod.addMVar(len(Q),vtype=['C']*(m+1)+['B']*m, lb=lb) 
ndxMod_con = ndxMod.addMConstrs(A, ndxMod_x, sense, b)
ndxMod.setMObjective(Q, C, 0, sense=gp.GRB.MINIMIZE)
ndxMod.Params.OutputFlag = 0
ndxMod.setParam('TimeLimit', RUNTIME)
ndxMod.optimize()

# save the coefficients
coef = ndxMod_x.x[:m+1]
        
print(coef)
### Make predictions of y_test using the saved coefficients
test = pd.read_csv('test_data.csv')
X_test = np.array(test.iloc[0:,1:])
y_test = np.array(test['y'])

n = X_test.shape[0] # number of rows
m = X_test.shape[1] # number of variables

# X variables for each record
X = np.zeros((n,m+1))
X[0:n, 0] = 1
for i in range(n):        
    X[i, 1:] = X_test[i,:]
    
X @ coef
### Calaculate the MSE of y_test
print('The MSE is:', mean_squared_error(y_test, X @ coef))
# Indirect Variable Selection – LASSO
### Fit a linear regression model using Lasso 
train = pd.read_csv('training_data.csv')
X_train = np.array(train.iloc[0:,1:])
y_train = np.array(train['y'])

lasso = linear_model.LassoCV(cv=10).fit(X_train, y_train)

print('Alpha value:', lasso.alpha_)
print('Model intercept:', lasso.intercept_)
print('Total X variables:', (lasso.coef_ != 0).sum())
print('X variable coefficients:', lasso.coef_)
### Make predictions of y_test
test = pd.read_csv('test_data.csv')
X_test = np.array(test.iloc[0:,1:])
y_test = np.array(test['y'])

lasso.predict(X_test)
### Calculate the MSE of y_test
print('The MSE is:', mean_squared_error(y_test, lasso.predict(X_test)))
### EDA of comparison
# predictions of y_test

test = pd.read_csv('test_data.csv')
X_test = np.array(test.iloc[0:,1:])
y_test = np.array(test['y'])

#lasso.predict(X_test)

sns.kdeplot(X@coef,alpha=0.35,fill='orange',label='MIQP',legend=True)
sns.kdeplot(lasso.predict(X_test),alpha=0.35,fill='skyblue',label='Lasso',legend=True)
plt.legend()
plt.xlabel('Prediction')plt.plot(coef[1:],alpha=0.55,label='MIQP',marker='.', markersize=10) 
plt.plot(lasso.coef_,alpha=0.55,label='Lasso',marker='.', markersize=10) #lets plot the second line
plt.ylabel('Feature Index')
plt.xlabel('Coefficient Value')
plt.title('Comparison of regression coefficients')
plt.legend(loc = 'best')
plt.show()